# 1.读取数据，分词，去除停用词

## 1.1读取数据
* weibo：DataFrame存储的博文及其对应标签；
* content：list，元素为各个博文，逗号分隔，博文为字符串形式。

In [1]:
import pandas as pd

In [2]:
weibo = pd.read_csv('./data/all_data.txt',sep='\t', names=['is_not_rumor','content'],encoding='utf-8')
weibo = weibo.dropna()#删除缺失值
weibo.head()

,is_not_rumor,content
0,0,2013年3月5日，《明报》做了题目为《如果有来生，你愿不愿意再做中国人？》的投票调查，截止...
1,1,各位同学，有认识住在华昌路靠近中兴路的海伦新苑的朋友吗？转给他们看一下好伐……帮在下留意一下...
2,1,#北京曝光# @倍儿小爽 ：今天回家路上接到老妈电话，老妈冒雨给我送饭，拿到饭我让老妈打车回...
3,0,一位可怜的94岁老奶奶，老伴去世，儿子伤寒死了，两个孙子在外面打工，她每天依靠捡垃圾为生，每...
4,1,有人好奇黑人妹子是如何化妆的么？LZ带图详解 [哈哈] 太搞笑了，一直以为他们是不化妆的～～（转）


In [3]:
weibo.shape

(3387, 2)

In [4]:
#将DataFrame中的Series转换为list
content = weibo.content.values.tolist()

In [5]:
print (content[3:5])

['一位可怜的94岁老奶奶，老伴去世，儿子伤寒死了，两个孙子在外面打工，她每天依靠捡垃圾为生，每天捡垃圾捡到凌晨2点，也只能转5、6元。奶奶双腿已经裂开，因为没钱，一直没有医治。求扩散~~每转一条微博，腾讯公益就像老奶奶捐出1毛钱，多转几次吧，不会脏了你微博，对么？', '有人好奇黑人妹子是如何化妆的么？LZ带图详解 [哈哈] 太搞笑了，一直以为他们是不化妆的～～（转）']


## 1.2分词
使用jieba分词器进行分词
* content_S：list of list，各个博文的分词结果；
* df_content：将content_S转换为DataFrame格式。

In [6]:
import jieba

In [7]:
#其元素为所有博文的分词结果，list类型
content_S = []
for line in content:
    current_segment = jieba.lcut(line)#列表，元素为分割出来的词
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        content_S.append(current_segment)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 1.867 seconds.
Prefix dict has been built successfully.


In [8]:
content_S[4]

['有人',
 '好奇',
 '黑人',
 '妹子',
 '是',
 '如何',
 '化妆',
 '的',
 '么',
 '？',
 'LZ',
 '带图',
 '详解',
 ' ',
 '[',
 '哈哈',
 ']',
 ' ',
 '太',
 '搞笑',
 '了',
 '，',
 '一直',
 '以为',
 '他们',
 '是',
 '不',
 '化妆',
 '的',
 '～',
 '～',
 '（',
 '转',
 '）']

In [9]:
df_content=pd.DataFrame({'content_S':content_S})
df_content.head()

,content_S
0,"[2013, 年, 3, 月, 5, 日, ，, 《, 明报, 》, 做, 了, 题目, 为..."
1,"[各位, 同学, ，, 有, 认识, 住, 在, 华昌路, 靠近, 中兴路, 的, 海伦, ..."
2,"[#, 北京, 曝光, #, , @, 倍儿, 小爽, , ：, 今天, 回家路上, 接..."
3,"[一位, 可怜, 的, 94, 岁, 老奶奶, ，, 老伴, 去世, ，, 儿子, 伤寒, ..."
4,"[有人, 好奇, 黑人, 妹子, 是, 如何, 化妆, 的, 么, ？, LZ, 带图, 详..."


## 1.3去除停用词
* contents_clean：list of list，contents去除停用词之后的结果；
* all_words：所有的词，包含重复值；
* df_content_clean：DtaFrame格式的contents_clean；
* df_all_words：DtaFrame格式的all_words。

In [10]:
stopwords=pd.read_csv("./stopwords/stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
stopwords.head()

,stopword
0,!
1,""""
2,#
3,$
4,%


In [11]:
def drop_stopwords(contents,stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))#str()转换为字符串##记录所有line_clean中的词
        contents_clean.append(line_clean)
    return contents_clean,all_words
    #print (contents_clean)
        

contents = df_content.content_S.values.tolist()    
stopwords = stopwords.stopword.values.tolist()
contents_clean,all_words = drop_stopwords(contents,stopwords)

In [12]:
df_content_clean=pd.DataFrame({'contents_clean':contents_clean})
df_content_clean.head()

,contents_clean
0,"[2013, 明报, 做, 题目, 有来生, 愿不愿意, 做, 中国, 投票, 调查, 截止..."
1,"[同学, 住, 华昌路, 靠近, 中兴路, 海伦, 新苑, 朋友, 转给, 好伐, 帮, 留..."
2,"[北京, 曝光, , 倍儿, 小爽, , 回家路上, 接到, 老妈, 电话, 老妈, 冒..."
3,"[一位, 可怜, 岁, 老奶奶, 老伴, 去世, 儿子, 伤寒, 死, 两个, 孙子, 外面..."
4,"[有人, 好奇, 黑人, 妹子, 化妆, LZ, 带图, 详解, , , 太, 搞笑, ..."


In [13]:
df_all_words=pd.DataFrame({'all_words':all_words})
df_all_words.head()

,all_words
0,2013
1,明报
2,做
3,题目
4,有来生


## 1.4统计词频，绘制词云图

In [ ]:
words_count=df_all_words
words_count['count']=df_all_words['all_words'].apply(lambda x: dict(df_all_words['all_words'].value_counts())[x])
words_count.drop_duplicates(["all_words"],inplace=True)
words_count.head()

In [ ]:
from wordcloud import WordCloud

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

%matplotlib inline

In [ ]:
wordcloud=WordCloud(font_path="./data/simhei.ttf",background_color="white",max_font_size=80)
word_frequence = {x[0]:x[1] for x in words_count.head(100).values}
wordcloud=wordcloud.fit_words(word_frequence)
plt.imshow(wordcloud)

# 2.分类检测
贝叶斯分类器

## 2.1划分数据集，数据预处理

In [14]:
df_train=pd.DataFrame({'contents_clean':contents_clean,'label':weibo['is_not_rumor']})
df_train.tail()

,contents_clean,label
3382,"[作品, 全职, 高手, 发布, 新, 章节, 一千六百, 三章, , 爱, 力量, 本书...",1
3383,"[微博报, 讯, 东方, 早报, 报道, 财报, 显示, 南方, 报业, 传媒, 集团, 一...",1
3384,"[天津, 北京, 山东, 江苏, 发出, 警示, 请, 传出去, , 隐翅虫, 青腰, 虫...",0
3385,"[俄媒, 如受, 制裁, , 俄罗斯, 立法, 没收, 欧美, 企业, 俄, 资产, 吃惊...",1
3386,"[盘锦市, 公安局, 查, 村民, 王树杰, 警察, 张研, 开枪, 杀害, 调查结果, 却...",0


In [15]:
## 划分数据集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['contents_clean'].values, df_train['label'].values, random_state=1)

In [16]:
x_train[-1]

['泰森',
 '赌城',
 '讲',
 '相声',
 '遭热',
 '捧',
 '法新社',
 '报道',
 '岁',
 '前',
 '世界',
 '拳王',
 '泰森',
 '昨晚',
 '拉斯维加斯',
 '米高梅',
 '酒店',
 '剧场',
 '开启',
 '单口相声',
 '演艺',
 '生涯',
 '历时',
 '两',
 '小时',
 '首演',
 '中',
 '泰森以',
 '夸张',
 '口吻',
 '噱头',
 '百出',
 '讲述',
 '穷小子',
 '拳王',
 '强奸犯',
 '直至',
 '一贫如洗',
 '经历',
 '现场',
 '观众',
 '对泰森',
 '报以',
 '热',
 '捧',
 '称其',
 '教育',
 '程度',
 '有限',
 '极具',
 '讲演',
 '天赋']

In [17]:
words = []
for line_index in range(len(x_train)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index,word_index)
words[-1]        

'泰森 赌城 讲 相声 遭热 捧 法新社 报道 岁 前 世界 拳王 泰森 昨晚 拉斯维加斯 米高梅 酒店 剧场 开启 单口相声 演艺 生涯 历时 两 小时 首演 中 泰森以 夸张 口吻 噱头 百出 讲述 穷小子 拳王 强奸犯 直至 一贫如洗 经历 现场 观众 对泰森 报以 热 捧 称其 教育 程度 有限 极具 讲演 天赋'

## 2.2贝叶斯分类器分类

In [18]:
from sklearn.naive_bayes import MultinomialNB

### 使用CountVectorizer作词嵌入

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer='word', max_features=4000,  lowercase = False)
#为words中每个词建立字典对应
vec.fit(words)

CountVectorizer(lowercase=False, max_features=4000)

In [20]:
classifier = MultinomialNB()
classifier.fit(vec.transform(words), y_train)

MultinomialNB()

In [21]:
# 测试集数据处理
test_words = []
for line_index in range(len(x_test)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        test_words.append(' '.join(x_test[line_index]))
    except:
         print (line_index,word_index)
test_words[1]

'晚安 心   Bernadette s   Song 小伙伴 生活 爆炸 最新 Howard 写给 Bernadette 那首 If   I   Didn t   Have   You 戳 泪点 首歌 作者 客串 Raj 前女友 Lucy Kate   Micucci   写 小伙伴 请 才华 鼓掌 欣赏 Bernadette s   Song http t cn zR6vrLS'

In [22]:
classifier.score(vec.transform(test_words), y_test)

0.8630460448642266

### 使用TfidfVectorizer作词嵌入

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vectorizer.fit(words)

TfidfVectorizer(lowercase=False, max_features=4000)

In [24]:
classifier1 = MultinomialNB()
classifier1.fit(vectorizer.transform(words), y_train)

MultinomialNB()

In [25]:
classifier1.score(vectorizer.transform(test_words), y_test)

0.8748524203069658